# Cats vs Dogs

Chúng ta sẽ tạo một thuật toán phân biệt mèo và chó thông qua hình ảnh

Với con người, việc phân biệt giữa chó và mèo là một việc vô cùng đơn giản nhưng đối với một máy tính thì gặp khá nhiều khó khăn.

![](https://storage.googleapis.com/kaggle-competitions/kaggle/3362/media/woof_meow.jpg)

# Khai báo thư viện

Chúng ta sẽ khai báo tất cả các thư viện cần thiết và các hàm trong tensorflow để xây dựng và train mô hình deep learning

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import matplotlib.cm as cm
import os                  
import shutil              
from pathlib import Path
import random
import cv2

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.imagenet_utils import decode_predictions


In [ ]:
# Kiểm tra phiên bản của python và tensorflow
!python --version
print("TensorFlow:",tf.__version__)

In [ ]:
!nvidia-smi


Chúng ta sẽ thiết lập kích thước mong muốn của tấm ảnh. Mỗi tấm ảnh đều sẽ được cắt về kích thước đã thiết lập trước khi train

In [ ]:
HEIGHT = 224
WIDTH = 224

# Download hình ảnh chó và mèo

Chúng ta sẽ download hình ảnh chó mèo từ link này

In [ ]:
!wget 'https://s3.amazonaws.com/fast-ai-sample/dogscats.tgz'

Chúng ta sẽ giải nén file .TGZ bằng câu lệnh này

In [ ]:
!tar -xzf dogscats.tgz

# Kiểm tra dữ liệu



Đây là đường dẫn đến hai thư mục chứa hình chó và mèo

In [ ]:
DATA_PATH = Path('./dogscats')
dog_paths = [str(i) for i in (DATA_PATH/'train/dogs').glob('*') if i.is_file()]
cat_paths = [str(i) for i in (DATA_PATH/'train/cats').glob('*') if i.is_file()]

Đây là 5 đường dẫn của 5 tấm đầu tiên của thư mục chó

In [ ]:
dog_paths[:5]

... và 5 tấm đầu tiên của thư mục mèo

In [ ]:
cat_paths[:5]


Lưu ý rằng những tấm ảnh này không được lưu trên máy tính mà được lưu tạm thời ở Google Colab

Chúng ta cũng có thể kiểm tra số lượng hình ảnh chó và mèo mà có thể dùng để train

In [ ]:
print(f"Number of dog images in our train dataset: {len(dog_paths)}")
print(f"Number of cat images int our train dataset: {len(cat_paths)}")

Chúng ta xem trong thư viện các ảnh mà ta có thể lấy ra được bằng câu lệnh```show_gallery```

In [ ]:
#@title
def reset_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
reset_seed()

def get_side(img, side_type, n = 5):
    h, w, c = img.shape
    if side_type == "horizontal":
        return np.ones((h,n,c))
    return np.ones((n,w,c))

def show_gallery(im_ls,n=5, shuffle=True):
    '''
    Adapted with serveral modifications 
    from https://www.kaggle.com/serkanpeldek/keras-cnn-transfer-learnings-on-cats-dogs-dataset
    '''
    images = []
    vertical_images = []
    if shuffle:
        random.shuffle(im_ls)
    vertical_images = []
    for i in range(n*n):
        img = load_img(im_ls[i], target_size=(HEIGHT,WIDTH))
        img = img_to_array(img)
        hside = get_side(img,side_type="horizontal")
        images.append(img)
        images.append(hside)
        
        if (i+1) % n == 0:
            himage=np.hstack((images))
            vside = get_side(himage, side_type="vertical")
            vertical_images.append(himage)
            vertical_images.append(vside)
            
            images = []
        
    gallery = np.vstack((vertical_images))
    plt.figure(figsize=(10,10))
    plt.axis("off")
    plt.imshow(gallery.astype(np.uint8))
    plt.show()

Giờ thì chúng ta có thể sử dụng ```show_gallery```
để hiện hình ảnh.

In [ ]:
# Show dog images
show_gallery(dog_paths, n=3)

In [ ]:
# Show cats images
show_gallery(cat_paths, n=3)

# Giờ thì chúng sẽ xây dựng và train mô hình deep learning

## Chuẩn bị dữ liệu cho mô hình bằng cách sử dụng Tensorflow 2


Chúng ta có thể train mô hình bằng GPU miễn phí từ Google. Tuy nhiên, trước xây dựng mô hình, các dữ liệu phải được chuẩn bị trước

Tất nhiên với deep learning, chúng ta sẽ không tải từng hình ảnh vào GPU (Vì số lượng ảnh rất lớn). Thay vào đó, chúng ta sẽ sử dụng **generator** để tải các hình ảnh khi được cần. Các hàm dưới đây sẽ giúp chúng ta tạo **generator**. Với **batch_size = 32**, chúng ta có thể khởi tạo 32 ảnh chúng một thời điểm

Một thứ thú vị khác mà chúng ta nên làm khi train mô hình deep learning là sử dụng một bộ dữ liệu khác gọi là **validation set**, thứ mà giúp chúng ta kiểm tra hiệu suất của mô hình (Chúng ta không train mô hình trên bộ dữ liệu này). Nếu hoạt động tốt trên bộ dữ liệu này nghĩa là mô hình chúng có thể hoạt động tốt ở trong tương lai

In [ ]:
def prepare_data(batch_size):
    train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

    train_generator = train_datagen.flow_from_directory(
                            DATA_PATH/'train', 
                            target_size=(HEIGHT,WIDTH),
                            batch_size=batch_size,
                            shuffle=True,
                            seed=42,
                            class_mode='categorical')
    
    validation_generator = val_datagen.flow_from_directory(
                            DATA_PATH/'valid',
                            target_size=(HEIGHT,WIDTH),
                            batch_size=batch_size,
                            shuffle=False,
                            class_mode='categorical')
    
    return train_generator,validation_generator


BATCH_SIZE = 32
train_generator,val_generator = prepare_data(BATCH_SIZE)

# Chọn kiến trúc deep learning

Ở bước này, chúng ta sẽ sử dụng mô hình đã được tạo sẵn, và được giải trong cuộc thi 2015 ImageNet Large Scale Visual Recognition. Mô hình này được gọi là ResNet, viết tắt cho Residual Networks, và nó được phát triển bởi Microsoft. Mặc dù được tạo trong năm 2015, nó được xem như là kiến trúc deep learning cơ bản vì được sử dụng cho các tác vụ thị giác máy tính. Một bước đột phá cơ bản của mô hình là **nó có thể dùng để train mạng nơ ron với hơn 100 lớp**


Đây là minh họa của kiến trúc này
![](https://images.viblo.asia/full/fe5b21e5-3ad3-4419-93e0-7aa77a662bdd.png)

In [ ]:
resnet = ResNet50V2(include_top=False, pooling="avg", weights='imagenet')
for layer in resnet.layers:
    layer.trainable=False

logits = Dense(2)(resnet.output)
output = Activation('softmax')(logits)
model = Model(resnet.input, output)


Chạy câu lệnh sau để hiểu thêm về mô hình deep learning

In [ ]:
model.summary()


Mỗi mô hình deep learning cần có một công cụ tối ưu hóa, chúng ta sẽ sử dụng công cụ tên là **Stochastic Gradient Descent**, thứ mà cho một thuật toán tối ưu để giảm thiểu **loss function**, giúp cải thiến hiệu suất của mô hình

In [ ]:
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)



Tại đây chúng ta sẽ có hai thông số là **loss function** và **accuracy**. **loss function** càng thấp và **accuracy** càng cao thì mô hình đang được train theo hướng ta mong muốn



In [ ]:
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])

Chúng ta sẽ cần một cái đánh dấu để lưu lại phiên bản tốt nhất của mô hình

In [ ]:
checkpointer = ModelCheckpoint(filepath="./resnet50best.h5", monitor='val_loss', save_best_only=True, mode='auto')



Chúng ta sẽ cho mô hình train toàn bộ dữ liệu ba lần

In [ ]:
history = model.fit(
    train_generator, 
    epochs=3,
    validation_data=val_generator,
    callbacks=[checkpointer]
)

# GradCAM 


Mô hình deep learning có thể có các tỉ lệ không lường trước được. Như vậy chúng ta cần sử dụng GradCAM để có thể xác định được mô hình dựa vào chi tiết nào để đưa ra kết quả


In [ ]:
#@title Run this cell to load GradCAM

class GradCAM:
    ''' 
    Adapted with some modification 
    from https://www.pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/
    '''
    def __init__(self, model, layerName=None):
        """
        model: pre-softmax layer (logit layer)
        """
        self.model = model
        self.layerName = layerName
            
        if self.layerName == None:
            self.layerName = self.find_target_layer()
    
    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM")
            
    def compute_heatmap(self, image, classIdx, upsample_size, eps=1e-5):
        gradModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output, self.model.output]
        )
        # record operations for automatic differentiation
        
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOuts, preds) = gradModel(inputs) # preds after softmax
            loss = preds[:,classIdx]
        
        # compute gradients with automatic differentiation
        grads = tape.gradient(loss, convOuts)
        # discard batch
        convOuts = convOuts[0]
        grads = grads[0]
        norm_grads = tf.divide(grads, tf.reduce_mean(tf.square(grads)) + tf.constant(eps))
        
        # compute weights
        weights = tf.reduce_mean(norm_grads, axis=(0,1))
        cam = tf.reduce_sum(tf.multiply(weights, convOuts), axis=-1)
        
        # Apply reLU
        cam = np.maximum(cam, 0)
        cam = cam/np.max(cam)
        cam = cv2.resize(cam, upsample_size,interpolation=cv2.INTER_LINEAR)
        
        # convert to 3D
        cam3 = np.expand_dims(cam, axis=2)
        cam3 = np.tile(cam3, [1,1,3])
        
        return cam3
    
def overlay_gradCAM(img, cam3):
    cam3 = np.uint8(255*cam3)
    cam3 = cv2.applyColorMap(cam3, cv2.COLORMAP_JET)
    
    new_img = 0.3*cam3 + 0.5*img
    
    return (new_img*255.0/new_img.max()).astype("uint8")


@tf.custom_gradient
def guidedRelu(x):
    def grad(dy):
        return tf.cast(dy>0,"float32") * tf.cast(x>0, "float32") * dy
    return tf.nn.relu(x), grad

class GuidedBackprop:
    def __init__(self,model, layerName=None):
        self.model = model
        self.layerName = layerName
        self.gbModel = self.build_guided_model()
        
        if self.layerName == None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if len(layer.output_shape) == 4:
                return layer.name
        raise ValueError("Could not find 4D layer. Cannot apply Guided Backpropagation")

    def build_guided_model(self):
        gbModel = Model(
            inputs = [self.model.inputs],
            outputs = [self.model.get_layer(self.layerName).output]
        )
        layer_dict = [layer for layer in gbModel.layers[1:] if hasattr(layer,"activation")]
        for layer in layer_dict:
            if layer.activation == tf.keras.activations.relu:
                layer.activation = guidedRelu
        
        return gbModel
    
    def guided_backprop(self, images, upsample_size):
        """Guided Backpropagation method for visualizing input saliency."""
        with tf.GradientTape() as tape:
            inputs = tf.cast(images, tf.float32)
            tape.watch(inputs)
            outputs = self.gbModel(inputs)

        grads = tape.gradient(outputs, inputs)[0]

        saliency = cv2.resize(np.asarray(grads), upsample_size)

        return saliency

def deprocess_image(x):
    """Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    """
    # normalize tensor: center on 0., ensure std is 0.25
    x = x.copy()
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.25

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
#@title Run this cell to prepare for GradCAM visualization

def show_gradCAMs(model, gradCAM, GuidedBP, im_ls, n=3, decode={}):
    """
    model: softmax layer
    """
    random.shuffle(im_ls)
    plt.subplots(figsize=(15, 5*n))
    k=1
    for i in range(n):
        img = cv2.imread(im_ls[i])
        upsample_size = (img.shape[1],img.shape[0])
        # Show original image
        plt.subplot(n,3,k)
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        plt.title("Filename: {}".format(im_ls[i]), fontsize=10)
        plt.axis("off")
        # Show overlayed grad
        plt.subplot(n,3,k+1)
        im = img_to_array(load_img(im_ls[i], target_size=(HEIGHT,WIDTH)))
        x = np.expand_dims(im, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        idx = preds.argmax()
        if len(decode)==0:
            res = decode_predictions(preds)[0][0][1:]
        else:
            res = [decode[idx],preds.max()]
        cam3 = gradCAM.compute_heatmap(image=x, classIdx=idx, upsample_size=upsample_size)
        new_img = overlay_gradCAM(img, cam3)
        new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
        plt.imshow(new_img)
        plt.title("GradCAM - Pred: {}. Prob: {}%".format(res[0],round(res[1]*100,2)), fontsize=10)
        plt.axis("off")
        
        # Show guided GradCAM
        plt.subplot(n,3,k+2)
        gb = GuidedBP.guided_backprop(x, upsample_size)
        guided_gradcam = deprocess_image(gb*cam3)
        guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
        plt.imshow(guided_gradcam)
        plt.title("Guided GradCAM", fontsize=10)
        plt.axis("off")
        
        k += 3
    plt.show()

Tải phiên bản tốt nhất mô hình

In [ ]:
model.load_weights("./resnet50best.h5")

... và tạo sự chuẩn bị cho GradCAM Visualization

In [ ]:
model_logit = Model(model.input,model.layers[-2].output)
gradCAM = GradCAM(model=model_logit, layerName="conv5_block3_out")
guidedBP = GuidedBackprop(model=model,layerName="conv5_block3_out")

In [ ]:

predictions = model.predict(val_generator, verbose = 1)
pred_indices = np.argmax(predictions,axis=1)

ground_truth = val_generator.classes
filenames = np.array(val_generator.filenames)

## Phần nào của tấm ảnh được mô hình dùng để dự đoán chó?

Chúng ta sẽ lấy tất cả các dự đoán đúng của chó

In [ ]:
dogs_correct_path = filenames[(pred_indices == ground_truth) & (ground_truth==1)]
dogs_correct_path = list(map(lambda x: str(DATA_PATH/'valid'/x),dogs_correct_path))


Và cách mà mô hình dự đoán rằng đó là một con chó

In [ ]:
show_gradCAMs(model, gradCAM,guidedBP,dogs_correct_path, n=5, decode={0:"cat", 1:"dog"})

## Phần nào của tấm ảnh được mô hình dùng để dự đoán mèo?

Chúng ta sẽ lấy tất cả các dự đoán đúng của mèo

In [ ]:
cats_correct_path = filenames[(pred_indices == ground_truth) & (ground_truth==0)]
cats_correct_path = list(map(lambda x: str(DATA_PATH/'valid'/x),cats_correct_path))


Và cách mà mô hình dự đoán rằng đó là một con mèo

In [ ]:
show_gradCAMs(model, gradCAM,guidedBP,cats_correct_path, n=5, decode={0:"cat", 1:"dog"})

# Các dự đoán sai

## Các bức ảnh này là của chó nhưng mô hình lại dư đoán là mèo



In [ ]:
cat_but_actually_dog_path = filenames[(pred_indices != ground_truth) & (ground_truth==1)]
cat_but_actually_dog_path = list(map(lambda x: str(DATA_PATH/'valid'/x),cat_but_actually_dog_path))

In [ ]:
show_gradCAMs(model, gradCAM,guidedBP,cat_but_actually_dog_path, n=3, decode={0:"cat", 1:"dog"})

## Các bức ảnh này là của mèo nhưng mô hình lại dư đoán là chó



In [ ]:
dog_but_actually_cat_path = filenames[(pred_indices != ground_truth) & (ground_truth==0)]
dog_but_actually_cat_path = list(map(lambda x: str(DATA_PATH/'valid'/x),dog_but_actually_cat_path))

In [ ]:
show_gradCAMs(model, gradCAM,guidedBP,dog_but_actually_cat_path, n=3, decode={0:"cat", 1:"dog"})

# Kiểm tra bằng hình ảnh của bạn

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  uploaded_path = '/content/' + fn
  print(f'You have uploaded this image file: {uploaded_path}' )

Kiểm tra xem mô hình đoán ảnh của bạn là chó hay mèo

In [ ]:
show_gradCAMs(model, gradCAM,guidedBP,[uploaded_path], n=1, decode={0:"cat", 1:"dog"})